# Grid Worlds

We are exploring some grid world problems trhough the eyes of Reinforcement Learning and Dynamic Programing. So lets see what this means.

## the world and the agent: states and actions


A grid world for us, is an $n \times m$ grid, where each pair of coordinates represents a position, which we will call a **state**. 

The agent is like a little robot that finds itself in this world. It will move through the world by selecting **actions**(up, down, left, right and the diagonals if we want to include then).

The world will contain non-empty spaces:

* initial state: a state where the agent will start
* terminal states: one or more states the agent is trying to reach
* walls: spaces the agent can't enter
* traps: a state the agent should avoid, and that will send it to the initial state

The goal of our agents will be to reach the initial state as fast as possible, avoiding the traps.

Bellow is an example of a world.

In [1]:
from notebooks.utils.worlds import small_world_03
from grid_world.visualization.format_objects import get_world_str

print(get_world_str(small_world_03))

6                ✘ 

5 ☠  ☠     █     █ 

4          █       

3    ☠  ☠  █  █    

2                  

1    █  █  █  █    

0 ⚐                

  0  1  2  3  4  5 


To add some stochasticity we can also add a "wind" to the world. The wind will add at the desired states a probability of pushing the agent in a given direction. So for instance if the agent is at state (1, 1), which has an wind going up with probability 0.1, and selects action left, it will have probability 0.9 of going to (0, 1) and 0.1 of going to (0, 2), assuming both states are empty.

We will use $S$ to denote the set of possible states and $A$ the set of actions.

## effects and rewards

Every time our agent takes an action the world sends a signal back, which we are calling the **effect**; this signal informs if the agent was inside a trap(-1), inside the terminal state(1) or in an empty state(0). This signal is what the agent is suposed to use to guide its behaviour, i.e. try to learn whether his actions were good or not. We will denote it by

$$ E: S \rightarrow \mathbb{Z} $$

Based on this action our agent will determine a **reward**, so if the agent receives a desired effect it should determine a larger reward then it would for a undesired one. We are treating the reward as something internal to the agent, as oposed to the effect, which is something he receives from the world. The idea here is that different rewards can determine different behaviours for the agent in the same world, so we are viewing this as a change in the agent itself. Techinically the reward is a function from an effect to a float(we will denote this as $R_0$), however we will usually think of the function bellow when we are talking about rewards:

$$ R: S \rightarrow \mathbb{R} \\ R(s) = R_0(E(s))$$

Notice that $E$ is deterministic, since it only depends on the state the agent is taking an action, which means our reward will also be deterministic.

Most of the time we will use the reward determined by the function below, we call it the "basic reward":

$$R_0(i) =    
     \begin{cases}
       -1 &\quad\text{if } i = 0 \\
       0 &\quad\text{if } i = 1 \\
       -100 &\quad\text{if } i = -1 \\
     \end{cases}
$$

## episodes and returns

An agent run trhough the world, from start to finish, is called an **episode**. We usually view episodes as a sequence of the form

$$s_0, a_0, r_0, s_1, a_1, r_1,  ... , s_T, a_T, r_T$$

where $s_t, a_t, r_t, s_{t+1}$ correspond accordingly to the state the agent is at time t, the action he choses, the reward he gets and then the new states he ends at. This is slightly different from the Sutton and Barto book, since we are using $r_t$ where they would use $r_{t+1}$(I will say I find their notation a little weird in this regard).

The reward $r_t$ may give an imediate idea of how good an action was, but in order to actually learn paths, which involve taking multiple actions in sequence we will need to look to ahead, to all the rewards we got in that episode. So in order to determine how good an episode is, or more generaly how well the agent did from time $t$ onwards, we use a cumulative function of the reward called the return:

$$G: \mathbb{Z} \to \mathbb{R} \\ G_t = \sum_{k = 0}^{\infty} \gamma^k r_{t+k} $$

Where $\gamma \in [0,1]$ is a parameter we must chose, and $r_t=0$ for $t>T$. For our problem we will usually set $\gamma = 1$; so we will essentially look at the sum of all rewards we get after selecting an action.

With this in place, we can think of the agent as trying to maxime the returns it will get in any given apisode.

## policies and value functions

The agent behaviour is specified by what we call a policy, usually denoted by $\pi$. The policy gives for each state $s$ a probability distribution over the actions $A$, i.e. $\pi: S \to \mathbb{P}(A)$. In practice we just define
$$ \pi: S \times A \to [0,1] \\
\text{satisfaying }\sum_{a \in A}\pi(a) = 1 $$

now given a world and a policy we can see that there is an expected return we will get by following this policy from a given state. This is called a value function 

$$ 
v_\pi: S \to \mathbb{R} \\
v_\pi(s) = \mathbb{E}_\pi(G_t | s_t = s )
$$

Here $\mathbb{E}_\pi$ denotes the expected value under policy $\pi$.

Very similar is another value function 
$$
Q_\pi: S \times A \to \mathbb{R} \\
Q_\pi(s, a) = \mathbb{E}_\pi(G_t | s_t = s \land a_t = a)
$$ 
which gives estimates for a state action pair.


## world models

Another ingredient we may use sometimes are world models. Mathematicaly this a function:

$$ M_w: S \times A \to \mathbb{P}(S) $$

where $M_w$ gives for each pair state action $(s,a)$ a probability distribution over the states $S$, that indicates the probabilitie of moving to this new state when taking action $a$ in state $s$. This means that $M_w(s,a): S \to [0, 1]$ is also a function and $M_w(s,a)(s_0)$ is the probability of getting to $s_0$ when taking action $a$ in state $s$.

Intuitively this a very descriptive function that gives us all the information about the dynamics of a world.

## Solutions

The solutions we will go through in this project involve learning a good policy $\pi$. This is usually done through a process called GPI, where we simultaneasly improving estimates for a $Q$ function and improving the policy $\pi$ based on this function. Since the police keeps changing as we go, the $Q$ function we will be updating isn't necessarily approximating a particular value function $Q_\pi$(especially because we don't what is the policy we are trying to achieve in the first place), nevertheless this is done in a smart way where we can get to good policies. Some details are povided in the specific notebooks, but to better understand what is going in GPI I highly recommend reading  ["Reinforcement Learning: An Introduction"](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).

The solutions we will explore can be viewed in two groups.

### Dynamic Programing

Dynamic Programing is great. It completly solves the problem, in the sense that it is capable to determine what is the optimal policy an agent should follow. It has a drawback though, we need to know a model of the world! So as great as it is, in some cases it just presuposes too much knowledge about the problem.

If we think about it, having a model of the world should indeed be enough to find an optimal solution. This doesn't mean that it is easy to find this solution, and having an algorithm capable of doing so is great. Our goal in the DP section is to explore such solution.

### Exploration based methods

There are a group of methods that I am calling "exploration" based methods. By these I mean any algorithm that resorts on a process like this: 

* an agent interacts with a world by taking actions and completing episodes
* it uses the information it gathers from these events to improve its policy

This is different in an essential way, as the agent is not given information on how the world works beforehand; it has to "go out there" and collect this information itself. 

The actions the agent takes during an episode affect both the returns it gets and the data it collects. This leads to a situation called the "exploration vs exploitation", where the agent will have to decide whether he wants to stay in what he believes is the best path, or go exploring for new information.

There are many solutions that fall under this group, including Monte Carlo methods and all variations of TD learning.